In [2]:

!pip install -q datasets evaluate


In [3]:
# Now we import all the required libs
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
import numpy as np
import evaluate


In [4]:
# Loading the NewsQA dataset from HF

dataset = load_dataset("lucadiliello/newsqa")

In [ ]:
# Now we load a pre-trained RoBERTa model that has already been trained on squad2
model_name = "deepset/roberta-base-squad2"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

C:\Users\LOCALACCOUNT\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\LOCALACCOUNT\.cache\huggingface\hub\models--deepset--roberta-base-squad2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

Exception in thread Thread-8:
Traceback (most recent call last):
  File "C:\Users\LOCALACCOUNT\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\LOCALACCOUNT\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\_monitor.py", line 84, in run
    instance.refresh(nolock=True)
  File "C:\Users\LOCALACCOUNT\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\std.py", line 1347, in refresh
    self.display()
  File "C:\Users\LOCALACCOUNT\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\notebook.py", line 171, in display
    rtext.value = right
    ^^^^^^^^^^^
  File "C:\Users\LOCALACCOUNT\AppData\Local\Programs\Python\Python312\Lib\site-packages\traitlets\traitlets.py", line 716, in __set__
    self.set(obj, value)
  File "C:\Users\LOCALACCOUNT\AppData\Local\Programs\Python\Python312\Lib\site-packages\traitlets\traitlets.py", line 706, in set
    obj._notify_trait(self.name, old_value,

In [ ]:
# This helper function is used to take the text and the answer start
# fields from the answers in the dataset
# Some samples might have missing values and stuff so handeling that tooo
def _extract_answer_info(answer_obj):
    if isinstance(answer_obj, dict):
        texts = answer_obj.get("text", [])
        starts = answer_obj.get("answer_start", [])
        if len(texts) == 0 or len(starts) == 0:
            return None, []
        return texts[0], starts
    if isinstance(answer_obj, list):
        if len(answer_obj) == 0:
            return None, []
        first = answer_obj[0]
        if isinstance(first, dict):
            texts = first.get("text", [])
            starts = first.get("answer_start", [])
            if len(texts) == 0 or len(starts) == 0:
                return None, []
            return texts[0], starts
    return None, []


In [ ]:
# It tokenizes both the question and the context
def preprocess_function(examples):
    questions = [q.strip() if q else "" for q in examples["question"]]
    contexts = [c if c else "" for c in examples["context"]]

    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation="only_second",
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    sample_mapping = inputs.pop("overflow_to_sample_mapping")
    offset_mapping = inputs.pop("offset_mapping")

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = inputs["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = inputs.sequence_ids(i)
        sample_index = sample_mapping[i]
        answer_obj = examples["answers"][sample_index]

        answer_text, answer_starts = _extract_answer_info(answer_obj)
        if answer_text is None or len(answer_starts) == 0:
            start_positions.append(cls_index)
            end_positions.append(cls_index)
            continue

        start_char = int(answer_starts[0])
        end_char = start_char + len(answer_text)

        token_start_index = 0
        while sequence_ids[token_start_index] != 1:
            token_start_index += 1
        token_end_index = len(input_ids) - 1
        while sequence_ids[token_end_index] != 1:
            token_end_index -= 1

        if not (start_char >= offsets[token_start_index][0] and end_char <= offsets[token_end_index][1]):
            start_positions.append(cls_index)
            end_positions.append(cls_index)
        else:
            while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                token_start_index += 1
            start_positions.append(token_start_index - 1)

            while token_end_index >= 0 and offsets[token_end_index][1] >= end_char:
                token_end_index -= 1
            end_positions.append(token_end_index + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


In [ ]:
# Since the dataset is large, we only take a small portion 
train_subset = dataset["train"].select(range(2000))
val_subset   = dataset["validation"].select(range(500))

# Applying the preprocessing function to both training and validation samples
tokenized_train = train_subset.map(preprocess_function, batched=True, remove_columns=train_subset.column_names)
tokenized_val   = val_subset.map(preprocess_function, batched=True, remove_columns=val_subset.column_names)


In [ ]:
# Trying to Hypertune it!
args = TrainingArguments(
    output_dir="roberta_qa_model",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=50,
    save_total_limit=1
)


In [ ]:
# Initializing the Trainer class which handles training and evaluation for us
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer
)


In [ ]:
trainer.train()


In [ ]:
#evaluvating it
metric = evaluate.load("squad")

def compute_metrics_from_logits(start_logits, end_logits, features, raw_examples):
    preds = []
    for i, (s_log, e_log) in enumerate(zip(start_logits, end_logits)):
        s = int(np.argmax(s_log))
        e = int(np.argmax(e_log))
        if s > e:
            pred_text = ""
        else:
            pred_text = tokenizer.decode(features["input_ids"][i][s:e+1], skip_special_tokens=True).strip()
        preds.append({"id": str(i), "prediction_text": pred_text})

    refs = []
    for i in range(len(raw_examples)):
        txt, starts = _extract_answer_info(raw_examples[i]["answers"])
        if txt is None:
            refs.append({"id": str(i), "answers": {"text": [], "answer_start": []}})
        else:
            refs.append({"id": str(i), "answers": {"text": [txt], "answer_start": [int(starts[0]) if len(starts)>0 else 0]}})
    return metric.compute(predictions=preds, references=refs)

predictions = trainer.predict(tokenized_val)
start_logits, end_logits = predictions.predictions
results = compute_metrics_from_logits(start_logits, end_logits, tokenized_val, val_subset)

print("Evaluation results:", results)


In [ ]:
# applying in the context
context = "Apple Inc. was founded by Steve Jobs, Steve Wozniak, and Ronald Wayne in April 1976."
question = "Who founded Apple Inc.?"

inputs = tokenizer(question, context, return_tensors="pt")
outputs = model(**inputs)

s = int(np.argmax(outputs.start_logits.detach().numpy()))
e = int(np.argmax(outputs.end_logits.detach().numpy()))

print("QA Test:", tokenizer.decode(inputs["input_ids"][0][s:e+1], skip_special_tokens=True))
